In [84]:
#! pip install -r requirements.txt

In [87]:
import re
from dotenv import load_dotenv
import streamlit as st
from PyPDF2 import PdfReader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import VertexAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.chains.question_answering import load_qa_chain
from langchain import PromptTemplate
from langchain.llms import VertexAI
from langchain.callbacks import get_openai_callback
from langchain.callbacks import context_callback
from google.cloud import aiplatform

load_dotenv()

aiplatform.init(project="vntg-mlops", location="us-west1")

In [48]:
!pip install faiss-cpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 2.2 MB/s eta 0:00:0000:0100:01
DEPRECATION: textract 1.6.5 has a non-standard dependency specifier extract-msg<=0.29.*. pip 23.3 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of textract or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063


In [92]:
path = "temp/HANBIT_Google_Cloud_Platform.pdf"
pdf_reader = PdfReader(path)
text = ""
for page in pdf_reader.pages:
    text += page.extract_text()

text = re.sub('[^가-힝0-9a-zA-Z\\s\(\)\.\:\/]', '', text)
text = text.replace("\r","")
text = re.sub('\n[0-9]*\n','',text)


In [93]:
len(text)

87795

In [94]:
text_splitter = CharacterTextSplitter(
    separator="\n",
    chunk_size=1000,
    chunk_overlap=200,
    length_function=len
)

chunks = text_splitter.split_text(text)

In [95]:
chunks[3]

'되었으면 좋겠다.\n최유석\n클라우드 서비스의 춘추 전국 시대라고 해도 과언이 아닐 정도로 많은 클라우드 서비\n스들이 생겨나고 있다. 하지만 서비스마다 사용법이 다르고 기존 IDC에서의 아키텍\n처 구성과는 많은 부분이 다르기에 클라우드를 처음 접하는 이들에게는 많은 부분이 \nIT 개발자를 위한 전자책 리얼타임 eBook(www.hanbit.co.kr)\n생소하고 어려울 수 있을 것이다. 내가 클라우드 서비스를 이용하기 위한 교육을 처음 \n받았을 때가 생각난다. 모든 문서는 영어였고 국내에는 도움을 청할 사람은 없었으며 \n업체에서의 지원은 미비하여 약 3개월을 혼자 밤새워가면서 서비스 아키텍처를 설계\n하고 구현해야 했다.\n구글이 자체적으로 사용하던 서비스를 구글 클라우드 플랫폼이라는 이름으로 서비\n스를 소개한 지 불과 몇 년 지나지 않은 거 같은데 Kubernetes 를 기반으로 하는 \nContainer  Engine 이나 자체 네트워크를 이용한 데이터센터 연동 등 다른 클라우\n드 서비스에는 없는 구글만의 장점으로 클라우드 서비스를 만들어 나가고 있다. 이 모습을 보면서 지난 해부터 클라우드를 처음 접하는 사람을 위한 책을 써보고 싶었는데 \n마침 이런 책을 만들 수 있는 기회가 주어져 매우 기쁜 마음으로 참여하게 되었다. \n윤성재\n구글 클라우드 플랫폼의 사용자로서 구글 클라우드에 입문하시고자 하는 사람들을 지면을 통해 만나게 되어 반갑다. 클라우드가 많은 관심을 받고 있지만 국내에서는 아직 낯설\n은 면이 많이 있다. 이 책을 통해 구글 클라우드를 시작하시는 사람들에게 조금이나마 도\n움이 될 수 있기를 희망한다. 구글 클라우드를 사용하는 데 어려움이 있다면 언제든지 연\n락 바라고 작성된 글에 대한 피드백을 주면 반영하여 계속 보완해보록 하겠다.\n김영균들어가며\n이 책은 구글 클라우드를 처음 사용하는 사람이 가상 머신 (VM) 기반의 클라우드 서비\n스 기능들을 빠르게 사용할 수 있도록 도울 목적으로 실습 위주로 속도감 있게 구성하\

In [49]:
embeddings = VertexAIEmbeddings(model_name="textembedding-gecko-multilingual@latest")
knowledge_base = FAISS.from_texts(chunks, embeddings)

In [66]:
llm = VertexAI(temperature=0, model_name="text-bison-32k", max_output_tokens=512)
prompt_template = """Answer the question as precise as possible using the provided context. If the answer is
                not contained in the context, say "answer not available in context" \n\n
                Context: \n {context}?\n
                Question: \n {question} \n
                Answer:
              """
prompt = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)

In [96]:
user_question="mysql 5.5 의 내부엔진은 뭐야?"
docs = knowledge_base.similarity_search(user_question)
chain = load_qa_chain(llm, chain_type="stuff", prompt=prompt)
response = chain.run(input_documents=docs, question=user_question)
print(response)

 InnoDB
